IMPORT FIRST DATASET

In [ ]:
from sklearn import datasets
import random as r
r.seed(30)
training20=datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# training20
#convert train text to feature vectors
import sklearn
# NGRAM CHANGE IT
count_vect = sklearn.feature_extraction.text.CountVectorizer(stop_words='english',ngram_range=(1,1))
# count_vect = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
X_train_counts = count_vect.fit_transform(training20.data)
#Convert train count vectors to tfidf
from scipy.sparse.lil import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
import numpy as np
#Convert test set to vectors to tfidf
twenty_test = datasets.fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
X_new_counts = count_vect.transform(twenty_test.data)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

IMPORT SECOND DATASET

In [ ]:
#import sentiment140 dataset into pandas
import pandas as pd
import csv
col=["sentiment", "tweetID", "date", "query", "user", "text"]
df=pd.read_csv("/content/training.1600000.processed.noemoticon.csv", 
               encoding='latin-1', engine='python', error_bad_lines=False, names=col)
from sklearn.utils import shuffle
df = shuffle(df,random_state=0)
sentiment_df = df["sentiment"]
text_df = df["text"]
sentiment = sentiment_df[:10000].to_numpy()
text = text_df[:10000].to_numpy()

#convert text to feature vectors
import sklearn
# from scipy.sparse.lil import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

# count_vect2 = sklearn.feature_extraction.text.CountVectorizer(stop_words='english',ngram_range=(1,1))
count_vect2 = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
X_train_counts2 = count_vect2.fit_transform(text)
# X_train_counts.shape

tfidf_transformer2 = TfidfTransformer()
X_train_tfidf2 = tfidf_transformer2.fit_transform(X_train_counts2)
# X_train_tfidf.shape
X_train_tfidf2

#import sentiment140 TEST set into pandas
import pandas as pd
import csv
col=["sentiment", "tweetID", "date", "query", "user", "text"]
df2=pd.read_csv("/content/new_testdata.manual.2009.06.14.csv", 
               encoding='latin-1', engine='python', error_bad_lines=False, names=col)
test_text = df2["text"]
test_sentiment = df2["sentiment"]

t_text = test_text.to_numpy()
t_sentiment = test_sentiment.to_numpy()

import numpy as np
X_new_counts2 = count_vect2.transform(t_text)
X_new_tfidf2 = tfidf_transformer2.transform(X_new_counts2)
# docs_test = twenty_test.data
# docs_test
X_new_tfidf2
# twenty_test.data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


<359x18731 sparse matrix of type '<class 'numpy.float64'>'
	with 4105 stored elements in Compressed Sparse Row format>

In [ ]:
df

,sentiment,tweetID,date,query,user,text
557138,0,2204444171,Wed Jun 17 02:14:00 PDT 2009,NO_QUERY,einmensch,wants to compete! i want hard competition! i w...
349381,0,2017152437,Wed Jun 03 07:56:34 PDT 2009,NO_QUERY,keithmorrison,It seems we are stuck on the ground in Amarill...
182051,0,1967043408,Fri May 29 18:52:13 PDT 2009,NO_QUERY,PunkieDory,where the f are my pinking shears? rarararrrar...
571236,0,2208721054,Wed Jun 17 09:32:48 PDT 2009,NO_QUERY,DYkEY_tYPE,0ff t0 tHE MEEtiN.. i HAtE WhEN PPl V0lUNtEER...
1339637,4,2018731586,Wed Jun 03 10:25:27 PDT 2009,NO_QUERY,BlueSmartiies,@ reply me pls
...,...,...,...,...,...,...
152315,0,1932709242,Tue May 26 22:02:34 PDT 2009,NO_QUERY,Greenskull,"@majornelson LOVING the zune, hating the lack ..."
963395,4,1827437924,Sun May 17 10:42:36 PDT 2009,NO_QUERY,adbruin,@kahilee Sounds like it was a blast! Have a sa...
117952,0,1827736749,Sun May 17 11:20:21 PDT 2009,NO_QUERY,la_discoteca,@marchingstars i feel bad for the zines i stil...
1484405,4,2067663968,Sun Jun 07 12:44:35 PDT 2009,NO_QUERY,J_Kaye,@ketadiablo - Just finished DUST AND MOONLIGHT...


Gaussian Naive Bayes implementation

In [ ]:
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import datasets
# np.random.seed(1234)


class GaussianNaiveBayes:
    
    def __init__(self):
        return
    
    def fit(self, x, y):
        N, D = x.shape
        #C = np.max(y) + 1
        #C=2
        unique=np.unique(y)
        C=len(unique)

        # one parameter for each feature conditioned on each class
        mu, sigma = np.zeros((C,D)), np.zeros((C,D))
        Nc = np.zeros(C) # number of instances in class c

        # for each class get the MLE for the mean and std
        c=0;
        for u in unique:
            x_c = x[y == u]                           #slice all the elements from class c

            Nc[c] = x_c.shape[0]                      #get number of elements of class c
            mu[c,:] = np.mean(x_c,0)                  #mean of features of class c
            sigma[c,:] = 1e-9 + np.std(x_c, 0)               #std of features of class c
            c+=1;
            #sigma[c%3,:] = np.std(x_c, 0)

            #Nc[c%3] = x_c.shape[0]                      #get number of elements of class c
            #mu[c%3,:] = np.mean(x_c,0)                  #mean of features of class c
            #sigma[c%3,:] = 1e-9 + np.std(x_c, 0)               #std of features of class c
            #sigma[c%3,:] = np.std(x_c, 0)
            
        self.mu = mu                                  # C x D
        self.sigma = sigma                            # C x D
        self.pi = (Nc)/(N)                        #Laplace smoothing (using alpha_c=1 for all c) you can derive using Dirichlet's distribution
        return self

def logsumexp(Z):                                                # dimension C x N
    Zmax = np.max(Z,axis=0)[None,:]                              # max over C
    log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=0))
    return log_sum_exp

def predict(self, xt):
    Nt, D = xt.shape
    # for numerical stability we work in the log domain
    # we add a dimension because this is added to the log-likelihood matrix 
    # that assigns a likelihood for each class (C) to each test point, and so it is C x N
    log_prior = np.log(self.pi)[:, None]
    # logarithm of the likelihood term for Gaussian 
    # the first two terms are the logarithm of the normalization term in the Gaussian and the final term is the exponent in the Gaussian. 
    # Notice that we are adding dimensions (using None) to model parameters and data to make this evaluation. 
    # The reason is that sigma and mu are C x D, while the data x is N x D. We operate on a C x N x D shape by increasing the number of dimensions when needed
    log_likelihood = -.5 * np.log(2*np.pi) - np.log(self.sigma[:,None,:]) -.5 * (((xt[None,:,:] - self.mu[:,None,:])/self.sigma[:,None,:])**2)
    # now we sum over the feature dimension to get a C x N matrix (this has the log-likelihood for each class-test point combination)
    log_likelihood = np.sum(log_likelihood, axis=2)
    # posterior calculation
    log_posterior = log_prior + log_likelihood
    posterior = np.exp(log_posterior - logsumexp(log_posterior))
    return np.argmax(posterior.T, 1)                                                  # dimension N x C

GaussianNaiveBayes.predict = predict

In [1]:
def evaluate_acc(model, X_train, y, X_test, y_test):
  model = model
  model.fit(X_train.toarray(), y)
  p=model.predict(X_test[:].toarray())
  accur=np.mean(p == y_test[:])
  return accur



In [ ]:
from math import floor
from scipy.sparse import vstack
def cross_validation_split(train):
  cv_list = [[],[],[],[],[]]
  cv_split = floor(train.shape[0]/5)
  for i in range(0,5):
    x_train = vstack((train[:cv_split*i,], train[cv_split*(i+1):cv_split*5,]))
    x_validation= train[cv_split*i:cv_split*(i+1),]
    cv_list[i].append(x_train)
    cv_list[i].append(x_validation)
  return cv_list


In [ ]:
from math import floor
def cross_validation_split_target(train):
  cv_list = [[],[],[],[],[]]
  cv_split = floor(train.shape[0]/5)
  for i in range(0,5):
    x_train= np.concatenate((train[:cv_split*i,], train[cv_split*(i+1):cv_split*5,]))
    x_validation= train[cv_split*i:cv_split*(i+1)]
    cv_list[i].append(x_train)
    cv_list[i].append(x_validation)
  return cv_list


In [ ]:
first_train_counts = cross_validation_split(X_train_counts)
first_target = cross_validation_split_target(training20.target)
first_train_tfidf = cross_validation_split(X_train_tfidf)

In [ ]:
second_train_counts = cross_validation_split(X_train_counts2)
second_train_tfidf = cross_validation_split(X_train_tfidf2)
second_target = cross_validation_split_target(sentiment)

K-Fold

In [ ]:
def k_foldCV(model,train,target):
  res = []
  for i in range(len(train)):
    m = model
    # print("A")
    print
    m.fit(train[i][0].toarray(),target[i][0])
    # print("B")
    y_prob = m.predict(train[i][1].toarray())
    # print("C")
    accuracy = np.mean(y_prob == target[i][1])
    res.append(accuracy)
    print(f'test accuracy: {accuracy}')
  print(f'Total accuracy: {np.mean(res)*100}')

K-Fold but doesn't crash

In [ ]:
from math import ceil
def k_fold_memory(model,train,target):
  res = []
  for i in range(len(train)):
    m = model
    # print("A")
    print
    m.fit(train[i][0].toarray(),target[i][0])
    # print("B")
    count = 0
    n=5
    length = ceil(len(target[i][1])/n) + 1
    # print(length)
    for j in range(1,length):  
      if j == length-1:
        y_prob = model.predict(train[i][1][(j-1)*n:].toarray())
        accuracy = np.sum(y_prob == target[i][1][(j-1)*n:])
        count += accuracy
      else:
        y_prob = model.predict(train[i][1][(j-1)*n:j*n].toarray())
        accuracy = np.sum(y_prob == target[i][1][(j-1)*n:j*n])
        count += accuracy
      # print(f'test accuracy: {accuracy}')
    # print("C")
    acc = count / len(target[i][1])
    res.append(acc)
    print(f'test accuracy: {acc}')
  print(f'Total accuracy: {np.mean(res)*100}')

In [ ]:
train = X_train_tfidf.toarray()
y = training20.target
doc, features = train.shape
# Prior
unique, counts = np.unique(y, return_counts=True)
prior = dict(zip(unique, counts))
vocab = {}
hm = {}
for i in prior:
  prior[i] /= doc
  hm[i] = {}
  vocab[i] = 0
  for j in range(features):
    hm[i][j] = 1
# Conditional probabilities
# Create a dictionary with every class in it
for j in range(len(train)):
  vocab[y[j]] += np.count_nonzero(train[j])
  # if j % 500 == 0:
  #   print(j)
  nz = np.nonzero(train[j])[0]
  for k in nz:
    hm[y[j]][k] += (train[j][k])
# for i in hm:
#   for j in hm[i]:
#     hm[i][j] /= (doc+vocab[i])

In [ ]:
for i in a:
  print(count_vect2.get_feature_names_out()[i])

good
just
love
thanks
day
http
lol
going
quot
time


In [ ]:
training20.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
for i in range(20):
  a = list(dict(sorted(hm[i].items(), key=lambda item: item[1], reverse=True)))[:1]
  print(f'{training20.target_names[i]}: {count_vect.get_feature_names_out()[a[0]]} - {hm[i][a[0]]}')

alt.atheism: god - 16.885143940439875
comp.graphics: graphics - 22.27839115174076
comp.os.ms-windows.misc: windows - 45.96148999171559
comp.sys.ibm.pc.hardware: drive - 24.232572025030475
comp.sys.mac.hardware: mac - 25.144832160160604
comp.windows.x: window - 27.7852812266701
misc.forsale: sale - 24.400559943390522
rec.autos: car - 37.74108433156238
rec.motorcycles: bike - 31.02666661344683
rec.sport.baseball: year - 18.142359608916326
rec.sport.hockey: game - 24.89440391749523
sci.crypt: key - 36.21139172170438
sci.electronics: use - 10.611051532699056
sci.med: msg - 14.736067824761248
sci.space: space - 33.71381166888871
soc.religion.christian: god - 45.85983208903894
talk.politics.guns: gun - 25.765047764124546
talk.politics.mideast: israel - 30.542671837490897
talk.politics.misc: people - 14.480029425502215
talk.religion.misc: god - 15.066749386133079


5-fold GaussianNB first dataset count vectorizer with stopwords

In [ ]:
k_fold_memory(GaussianNaiveBayes(),first_train_counts,first_target)

test accuracy: 0.6273209549071618
test accuracy: 0.6317418213969939
test accuracy: 0.6414677276746242
test accuracy: 0.638815207780725
test accuracy: 0.6304155614500442
Total accuracy: 63.39522546419097


5-fold GaussianNB first dataset tfidf with stopwords

In [ ]:
k_fold_memory(GaussianNaiveBayes(),first_train_tfidf,first_target)

test accuracy: 0.6282051282051282
test accuracy: 0.6264367816091954
test accuracy: 0.6423519009725907
test accuracy: 0.6370468611847923
test accuracy: 0.6361626878868258
Total accuracy: 63.40406719717065


Get rid of warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

5-fold GaussianNB second dataset count vectorizor no stopwords

In [ ]:
k_fold(GaussianNaiveBayes(),second_train_counts,second_target)

test accuracy: 0.5005
test accuracy: 0.5005
test accuracy: 0.489
test accuracy: 0.49
test accuracy: 0.497
Total accuracy: 49.53999999999999


5-fold GaussianNB second dataset tfidf no stopwords

In [ ]:
k_fold(GaussianNaiveBayes(),second_train_tfidf,second_target)

test accuracy: 0.5005
test accuracy: 0.5005
test accuracy: 0.489
test accuracy: 0.49
test accuracy: 0.497
Total accuracy: 49.53999999999999


5-fold GaussianNB second dataset count vectorizor with stopwords

In [ ]:
k_fold(GaussianNaiveBayes(),second_train_counts,second_target)

test accuracy: 0.5005
test accuracy: 0.5005
test accuracy: 0.489
test accuracy: 0.49
test accuracy: 0.497
Total accuracy: 49.53999999999999


5-fold GaussianNB second dataset count vectorizor with stopwords

In [ ]:
k_fold(GaussianNaiveBayes(),second_train_tfidf,second_target)

test accuracy: 0.5005
test accuracy: 0.5005
test accuracy: 0.489
test accuracy: 0.49
test accuracy: 0.497
Total accuracy: 49.53999999999999


5-fold MultinomialNB second dataset count vectorizor with stopwords

In [ ]:
k_fold(MultinomialNaiveBayes(),second_train_counts,second_target)

test accuracy: 0.7185
test accuracy: 0.713
test accuracy: 0.7115
test accuracy: 0.6915
test accuracy: 0.707
Total accuracy: 70.83


5-fold MultinomialNB second dataset tfidf with stopwords

In [ ]:
k_fold(MultinomialNaiveBayes(),second_train_tfidf,second_target)

test accuracy: 0.7255
test accuracy: 0.7165
test accuracy: 0.716
test accuracy: 0.7
test accuracy: 0.713
Total accuracy: 71.42000000000002


5-fold MultinomialNB second dataset count vectorizor no stopwords

In [ ]:
k_fold(MultinomialNaiveBayes(),second_train_counts,second_target)

test accuracy: 0.742
test accuracy: 0.7235
test accuracy: 0.7325
test accuracy: 0.714
test accuracy: 0.7345
Total accuracy: 72.93


5-fold MultinomialNB second dataset tfidf no stopwords

In [ ]:
k_fold(MultinomialNaiveBayes(),second_train_tfidf,second_target)

test accuracy: 0.7485
test accuracy: 0.734
test accuracy: 0.7385
test accuracy: 0.725
test accuracy: 0.7425
Total accuracy: 73.77


Testing (1,2) n-grams on small subset of second dataset

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_tfidf2[:1000].toarray(), sentiment[:1000])

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:252: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [ ]:
len(t_sentiment)

359

In [ ]:
from math import ceil
l = []
n=10
length = ceil(len(t_sentiment)/n) + 1
count = 0
for j in range(1,length):
  if j == length-1:
    y_prob = model.predict(X_new_tfidf2[(j-1)*n:].toarray())
    accuracy = np.sum(y_prob == t_sentiment[(j-1)*n:])
    count += accuracy
  else:
    y_prob = model.predict(X_new_tfidf2[(j-1)*n:j*n].toarray())
    accuracy = np.sum(y_prob == t_sentiment[(j-1)*n:j*n])
    count += accuracy
  # print(f'test accuracy: {accuracy}')
acc = count / 359
acc

0.49303621169916434

Result of (1,2) n-gram with MultinomialNB on second dataset with best hyperparameters: 62.40%

Result of (1,2) n-gram with GaussianNB on second dataset with best hyperparameters: 49.30%

Result of (1,1) n-gram with MultinomialNB on second dataset with best hyperparameters: 68.80%

Result of (1,1) n-gram with GaussianNB on second dataset with best hyperparameters: 49.30%

Testing (1,2) n-grams on small subset of first dataset

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_tfidf[:].toarray(), training20.target[:])

In [ ]:
l = []
n=10
for i in range(1,753):
  y_prob = model.predict(X_new_tfidf[(i-1)*n:i*n].toarray())
  accuracy = np.mean(y_prob == twenty_test.target[(i-1)*n:i*n])
  l.append(accuracy)
  print(f'test accuracy: {accuracy}')

test accuracy: 0.6
test accuracy: 0.4
test accuracy: 0.8
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.7
test accuracy: 0.3
test accuracy: 0.4
test accuracy: 0.4
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.9
test accuracy: 0.7
test accuracy: 0.8
test accuracy: 0.5
test accuracy: 0.7
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.8
test accuracy: 0.6
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.7
test accuracy: 0.7
test accuracy: 0.7
test accuracy: 0.3
test accuracy: 0.8
test accuracy: 0.9
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.5
test accuracy: 0.8
test accuracy: 0.4
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.3
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.5
test accuracy: 0.7
test accuracy: 0.6
test accuracy: 0.4
test accuracy: 0.7
test accuracy: 0.4
test accuracy: 0.3
test accuracy: 0.6
test accuracy: 0.4
test accuracy: 0.5
test accurac

Result of (1,2) n-gram with MultinomialNB on first dataset with best hyperparameters: 29.74%

Result of (1,2) n-gram with GaussianNB on first dataset with count and stopwords: 43.46%

Result of (1,2) n-gram with GaussianNB on first dataset with best hyperparameters: 44.60%

In [ ]:
#32
np.mean(l)

0.5502659574468085

In [ ]:
model = GaussianNaiveBayes()
y_prob = model.predict(X_new_tfidf2.toarray(), sentiment)
y_pred = np.argmax(y_prob, 1)
accuracy = np.sum(y_pred == t_sentiment)/y_pred.shape[0]
l.append(accuracy)
print(f'test accuracy: {accuracy}')

TypeError: ignored

Testing (1,1) n-gram on small subset (this is default) to compare to (1,2)

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_counts[:].toarray(), training20.target[:])

In [ ]:
model.sigma[:10]

array([[1.00000000e-09, 4.19319680e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [3.43337931e-01, 2.33178693e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [2.58368488e-01, 7.10659908e-02, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       ...,
       [2.67528495e-01, 3.48822200e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [1.28621302e-01, 1.21752454e-01, 5.77347050e-02, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [5.48420938e+00, 1.28950377e+00, 1.00000000e-09, ...,
        4.08929846e-02, 1.00000000e-09, 1.00000000e-09]])

sentiment dataset before adding constant

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_counts2.toarray(), sentiment)
p=model.predict(X_new_counts2[:].toarray())

accur=np.mean(p == t_sentiment[:]%3)
accur


0.5710306406685237

In [ ]:
model.sigma[:]

array([[0.03757633, 0.02460091, 0.02460091, ..., 0.        , 0.        ,
        0.        ],
       [0.01990468, 0.02814389, 0.        , ..., 0.01407613, 0.01407613,
        0.03147269]])

In [ ]:
p

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

sentiment dataset gaussian with count occurences accuracy 

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_counts2.toarray(), sentiment)
p=model.predict(X_new_counts2[:].toarray())

accur=np.mean(p == t_sentiment[:]%3)
accur


0.5710306406685237

In [ ]:
model.sigma[:]

array([[3.75763358e-02, 2.46009062e-02, 2.46009062e-02, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [1.99046852e-02, 2.81438939e-02, 1.00000000e-09, ...,
        1.40761333e-02, 1.40761333e-02, 3.14726940e-02]])

sentiment dataset gaussian with tfidf accuracy

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_tfidf2.toarray(), sentiment)
p=model.predict(X_new_tfidf2[:].toarray())

accur=np.mean(p == t_sentiment[:]%3)
accur


0.5598885793871866

sentiment dataset gaussian count with stop words

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_counts2.toarray(), sentiment)
p=model.predict(X_new_counts2[:].toarray())

accur=np.mean(p == t_sentiment[:]%3)
accur


0.5738161559888579

sentiment gaussian tfidf stop words

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_tfidf2.toarray(), sentiment)
p=model.predict(X_new_tfidf2[:].toarray())

accur=np.mean(p == t_sentiment[:]%3)
accur

0.5654596100278552

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train_counts[:].toarray(), training20.target[:])

In [ ]:
model.sigma[:10]

array([[1.00000000e-09, 4.19319680e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [3.43337931e-01, 2.33178693e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [2.58368488e-01, 7.10659908e-02, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       ...,
       [2.67528495e-01, 3.48822200e-01, 1.00000000e-09, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [1.28621302e-01, 1.21752454e-01, 5.77347050e-02, ...,
        1.00000000e-09, 1.00000000e-09, 1.00000000e-09],
       [5.48420938e+00, 1.28950377e+00, 1.00000000e-09, ...,
        4.08929846e-02, 1.00000000e-09, 1.00000000e-09]])

In [ ]:
p=model.predict(X_new_counts[:10].toarray())
accur=np.mean(p == twenty_test.target[:10])
accur

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in subtract


0.1

In [ ]:
l = []
n=10
for i in range(1,753):  
  y_prob = model.predict(X_new_counts[(i-1)*n:i*n].toarray())
  accuracy = np.mean(y_prob == twenty_test.target[(i-1)*n:i*n])
  l.append(accuracy)
  print(f'test accuracy: {accuracy}')

test accuracy: 0.5
test accuracy: 0.4
test accuracy: 0.7
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.2
test accuracy: 0.4
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.9
test accuracy: 0.7
test accuracy: 0.8
test accuracy: 0.6
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.4
test accuracy: 0.9
test accuracy: 0.5
test accuracy: 0.4
test accuracy: 0.5
test accuracy: 0.7
test accuracy: 0.8
test accuracy: 0.7
test accuracy: 0.4
test accuracy: 0.7
test accuracy: 0.9
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.4
test accuracy: 0.5
test accuracy: 0.8
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.3
test accuracy: 0.6
test accuracy: 0.5
test accuracy: 0.7
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.6
test accuracy: 0.3
test accuracy: 0.7
test accuracy: 0.5
test accuracy: 0.5
test accuracy: 0.6
test accuracy: 0.4
test accuracy: 0.4
test accurac

Result of (1,1) n-gram with MultinomialNB on first dataset with best hyperparameters: 39.79%

Result of (1,1) n-gram with GaussianNB on first dataset with count + stopwords: 41.89%

Result of (1,1) n-gram with GaussianNB on first dataset with best hyperparameters: 42.55%

In [ ]:
np.mean(l)

0.550531914893617

Gaussian Naive Bayes accuracy from first dataset

In [ ]:
(0.549466*7500+0.625*32)/7532

0.5497869091874668

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(X_train_tfidf, training20.target)
docs_test = X_new_tfidf
predicted = clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.4579129049389272

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(X_train_tfidf[:].toarray(), training20.target[:])
docs_test = X_new_tfidf
predicted = clf.predict(docs_test[0:].toarray())
np.mean(predicted == twenty_test.target[0:])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, training20.target)
docs_test = X_new_tfidf
predicted = clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.6779075942644716

In [ ]:
from sklearn import model_selection
model = GaussianNaiveBayes()
model.fit(X_train_tfidf2.toarray(), sentiment)
y_prob = model.predict(X_new_tfidf2.toarray())
y_pred = np.argmax(y_prob, 1)
accuracy = np.sum(y_pred == t_sentiment)/y_pred.shape[0]
print(f'test accuracy: {accuracy}')

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:252: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


test accuracy: 0.49303621169916434


In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(X_train_tfidf2.toarray(), sentiment)
docs_test = X_new_tfidf2
predicted = clf.predict(docs_test.toarray())
np.mean(predicted == t_sentiment)

0.5738161559888579

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf2, sentiment)
docs_test = X_new_tfidf2
predicted = clf.predict(docs_test)
np.mean(predicted == t_sentiment)

0.7520891364902507

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(X_train_tfidf2, sentiment)
docs_test = X_new_tfidf2
predicted = clf.predict(docs_test)
np.mean(predicted == t_sentiment)

0.7493036211699164

Multionomial Naive Bayes Implementation

In [ ]:
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
# np.random.seed(1234)


class MultinomialNaiveBayes:
    
    def __init__(self):
        return
    
    def fit(self, x, y):
      #Compute all the priors for each class: number of documents of one class divided by total number of documents
      #Calculate Conditional probabilities: number of each word in each class, do +1 on numerator and +num of unique words in denominator for smoothing
      #To predict a word, calculate probability of each class by doing the below:
      #Multiply prior of the class by conditional probability of each word being in said class in that document
      #Choose the max probability as the predicted class
      #Have to somehow do this in logarithms i have no clue lol
      train = x
      doc, features = train.shape
      # Prior
      unique, counts = np.unique(y, return_counts=True)
      prior = dict(zip(unique, counts))
      vocab = {}
      hm = {}
      for i in prior:
        prior[i] /= doc
        hm[i] = {}
        vocab[i] = 0
        for j in range(features):
          # hm[i][j] = 1
          hm[i][j] = 0
      # Conditional probabilities
      # Create a dictionary with every class in it
      for j in range(len(train)):
        vocab[y[j]] += np.count_nonzero(train[j])
        # if j % 500 == 0:
        #   print(j)
        nz = np.nonzero(train[j])[0]
        for k in nz:
          hm[y[j]][k] += (train[j][k])
      for i in hm:
        for j in hm[i]:
          # hm[i][j] /= (doc+vocab[i])
          hm[i][j] /= vocab[i]
      self.hm = hm
      self.prior = prior

      return self

def logsumexp(Z):                                                # dimension C x N
    Zmax = np.max(Z,axis=0)[None,:]                              # max over C
    log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=0))
    return log_sum_exp

def predict(self, xt):
    from math import log, exp, pow
    result = []
    first = list(self.hm.keys())[0]
    # for each line to predict
    for i in range(len(xt)):
      # if i % 5 == 0:
      #   print(i)
      # for each class it could be
      tmp = first
      best = float('-inf')
      nz = np.nonzero(xt[i])[0]
      for j in self.hm:
        # calculate probability of that line being that class
        # and then choose the class with highest probability
        prob = log(self.prior[j])
        for k in range(len(nz)):
          # print(xt[i][nz[k]] * log(self.hm[j][nz[k]]))
          if (self.hm[j][nz[k]] == 0.0):
              prob += log(1)
          else:
              prob += xt[i][nz[k]] * log(self.hm[j][nz[k]])
          # prob += log(math.pow(self.hm[j][nz[k]],xt[i][nz[k]]))
        if best < prob:
          best = prob
          tmp = j
      result.append(tmp)
    res = np.array(result)
    return res                                      # dimension N x C

MultinomialNaiveBayes.predict = predict

Second dataset MultinomialNB

In [ ]:
from sklearn import model_selection
model = MultinomialNaiveBayes()
limit_training_set2 = floor(X_train_tfidf2.shape[0]*0.8)
model.fit(X_train_tfidf2[:limit_training_set2].toarray(), sentiment[:limit_training_set2])
y_prob = model.predict(X_new_tfidf2.toarray())
accuracy = np.mean(y_prob == t_sentiment)
print(f'test accuracy: {accuracy}')

First Dataset MultinomialNB

In [ ]:
model = MultinomialNaiveBayes()
from math import floor
limit_training_set = floor(X_train_counts.shape[0]*1.0)
model.fit(X_train_counts[:limit_training_set].toarray(), training20.target[:limit_training_set])
y_prob = model.predict(X_new_counts.toarray())
accuracy = np.mean(y_prob == twenty_test.target)
print(f'test accuracy: {accuracy}')

In [ ]:
model = GaussianNaiveBayes()
from math import floor
#limit_training_set = floor(X_train_coun.shape[0]*1.0)
model.fit(X_train_tfidf[:].toarray(), training20.target[:])
y_prob = model.predict(X_new_tfidf.toarray())
accuracy = np.mean(y_prob == twenty_test.target)
print(f'test accuracy: {accuracy}')